In [1]:
import os
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
sys.path.append('../../codes')
from utils import video_utils

In [3]:
!mkdir "../../Regioes/sepetiba"
!mkdir "../../Regioes/sepetiba/frames"
!mkdir "../../Regioes/sepetiba/frames/all"
!mkdir "../../Regioes/sepetiba/frames/overlaped"
!mkdir "../../Regioes/sepetiba/dataFrames"
!mkdir "../../Regioes/sepetiba/dataFrames/all"
!mkdir "../../Regioes/sepetiba/dataFrames/overlaped"

J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba.
J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba/frames.
J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba/frames/all.
J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba/frames/overlaped.
J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba/dataFrames.
J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba/dataFrames/all.
J� existe uma subpasta ou um arquivo ../../Regioes/sepetiba/dataFrames/overlaped.


In [4]:
ALL_FRAMES_STRING = 'sepetiba/frames/all/'
OVERLAPED = '../../Regioes/sepetiba/dataFrames/overlaped/'

In [5]:
VIDEO_PATH = '../../Dataset/videos/sepetiba/'
VIDEO_NAMES = ['20210317_rectified_DJI_0073.avi','20210317_rectified_DJI_0074.avi']

In [10]:
videos = []
logs = []
for i in VIDEO_NAMES:
  videos.append(os.path.join(VIDEO_PATH,i))

In [11]:
videos_obj = []
videos_capture = []
for i in videos:
  videos_capture.append(cv2.VideoCapture(i))

## Logging phase_correlation

In [8]:
def scale_img(src, scale):
    # calculate new dimensions
    new_w = int(src.shape[1] * scale)
    new_h = int(src.shape[0] * scale)

    return cv2.resize(src, (new_w, new_h))
def phase_correlation(img, img_offset, scale=None):
    """Image translation registration by cross-correlation.
    It obtains an estimate of the cross-correlation peak by an FFT.
    It is very similar to `skimage.feature.register_translation`.
    However, our function runs faster because we restrict it to our application.
    Args:
        img (array): image.
        img_offset (array): offset image. Must have the same dim as `img`.
        scale (float, optional): If not `None`, rescale input images to run faster.
        Defaults to None.
    Returns:
        array:  shift vector (in pixels) required to register `img_offset` with
        `img`.  Axis ordering is consistent with numpy (e.g. Z, Y, X)
    """

    if img.shape != img_offset.shape:
        raise ValueError("Error: images must be same size")

    # if images in BGR, tranform to grayscal and use fft2
    # which is much faster than using fftn
    # if img.ndim > 2:
    #     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # if img_offset.ndim > 2:
    #     img_offset = cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY)

    img = np.float32(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    img_offset = np.float32(cv2.cvtColor(img_offset, cv2.COLOR_BGR2GRAY))

    if scale is not None:
        img = scale_img(img, scale)
        img_offset = scale_img(img_offset, scale)

    (x, y), c = cv2.phaseCorrelate(img, img_offset)

    # img_fft = np.fft.fft2(img)
    # img_offset_fft = np.fft.fft2(img_offset)

    # shape = img_fft.shape
    # midpoints = np.array([np.fix(axis_size / 2) for axis_size in shape])

    # R = img_fft * img_offset_fft.conj()
    # # R /= np.absolute(R)  # normalize give wrong results for large frames interval (???)
    # # (print('norm'))
    # cross_correlation = np.fft.ifft2(R)

    # shifts = np.unravel_index(np.argmax(np.absolute(cross_correlation)), shape)
    # shifts = np.array(shifts, dtype=np.float64)
    # shifts[shifts > midpoints] -= np.array(shape)[shifts > midpoints]

    if scale is not None:
        x /= scale
        y /= scale

    return (x, y), c

In [9]:
r_array = []
x = []
y = []
c = []
counter = 0
for i in videos_capture:
    frameB = i.read()
    while frameB[0]:
        counter += 1
        frameA = frameB
        frameB = i.read()
        if frameB[0]:
          results = phase_correlation(frameA[1], frameB[1], 0.4)
          r_array.append(results)
          x.append(results[0][0])
          y.append(results[0][1])
          c.append(results[1])

x_dataFrame = pd.DataFrame(x)
x_dataFrame.columns = ['x']
y_dataFrame = pd.DataFrame(y)
y_dataFrame.columns = ['y']
c_dataFrame = pd.DataFrame(c)
c_dataFrame.columns = ['c']
r_dataFrame = pd.concat([x_dataFrame, y_dataFrame, c_dataFrame], axis=1)
r_dataFrame.to_csv('phase_correlate.csv', index=False)

In [12]:
r_array = []
x = []
y = []
c = []
counter = 0
for i in videos_capture:
    frameB = i.read()
    while frameB[0]:
        counter += 1
        frameA = frameB
        frameB = i.read()
        if frameB[0]:
          results = phase_correlation(frameA[1], frameB[1])
          r_array.append(results)
          x.append(results[0][0])
          y.append(results[0][1])
          c.append(results[1])

x_dataFrame = pd.DataFrame(x)
x_dataFrame.columns = ['x']
y_dataFrame = pd.DataFrame(y)
y_dataFrame.columns = ['y']
c_dataFrame = pd.DataFrame(c)
c_dataFrame.columns = ['c']
r_dataFrame = pd.concat([x_dataFrame, y_dataFrame, c_dataFrame], axis=1)
r_dataFrame.to_csv('phase_correlate.csv', index=False)